#Importing All Dependencies

In [14]:
import numpy as np
import pandas as pd
import ast
import nltk
import pickle



Connecting Drive to Google Colab

In [15]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Reading Dataset from Drive

In [16]:
movies =pd.read_csv('/content/drive/MyDrive/Movies-Data/tmdb_5000_movies.csv')
credits =pd.read_csv('/content/drive/MyDrive/Movies-Data/tmdb_5000_credits.csv')

Merged Both Dataset 

In [ ]:
movies=movies.merge(credits,on='title')
movies.info()

In [19]:
movies = movies[['id','title','overview','genres','keywords','cast','crew']]
movies.head()


,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [20]:
movies.isnull().sum()

id          0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [ ]:
movies.dropna(inplace=True)

In [22]:
movies.duplicated().sum()

0

In [ ]:
movies.drop_duplicates(inplace=True)

In [24]:
def convert(obj):
    L=[]
    for i in ast.literal_eval (obj):
        L.append(i['name'])
    return L

In [ ]:
movies['genres'] = movies['genres'].apply(convert)

In [26]:
movies.head()

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [27]:
movies['keywords'] = movies['keywords'].apply(convert)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [28]:
def convert3(obj):
    L=[]
    counter =0
    for i in ast.literal_eval (obj):
        if counter !=3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

In [29]:
movies['cast'] = movies['cast'].apply(convert3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [30]:
def fetch_director(obj):
    L=[]
    for i in ast.literal_eval (obj):
      if i['job'] == 'Director':
        L.append(i['name'])
        break
    return L

In [31]:
 movies['crew'] = movies['crew'].apply(fetch_director)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
def collapse(L):
    L1 =[]
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [ ]:
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['overview'] = movies['overview'].apply(collapse)

In [34]:
movies.head()

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[I, n, , t, h, e, , 2, 2, n, d, , c, e, n, t, ...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[C, a, p, t, a, i, n, , B, a, r, b, o, s, s, a...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,"[A, , c, r, y, p, t, i, c, , m, e, s, s, a, g,...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,The Dark Knight Rises,"[F, o, l, l, o, w, i, n, g, , t, h, e, , d, e,...","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,John Carter,"[J, o, h, n, , C, a, r, t, e, r, , i, s, , a, ...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


In [35]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
new =movies[['id','title','tags']]
new.head()

,id,title,tags
0,19995,Avatar,"[I, n, , t, h, e, , 2, 2, n, d, , c, e, n, t, ..."
1,285,Pirates of the Caribbean: At World's End,"[C, a, p, t, a, i, n, , B, a, r, b, o, s, s, a..."
2,206647,Spectre,"[A, , c, r, y, p, t, i, c, , m, e, s, s, a, g,..."
3,49026,The Dark Knight Rises,"[F, o, l, l, o, w, i, n, g, , t, h, e, , d, e,..."
4,49529,John Carter,"[J, o, h, n, , C, a, r, t, e, r, , i, s, , a, ..."


In [37]:
new['tags'] = new['tags'].apply(lambda x:" ".join(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
!pip install nltk

In [40]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [41]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [42]:
new['tags'] = new['tags'].apply(stem)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [43]:
from sklearn.feature_extraction.text import  CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [44]:
vectors = cv.fit_transform(new['tags']).toarray()

In [ ]:
cv.get_feature_names()

In [46]:
from sklearn.metrics.pairwise import cosine_similarity

In [47]:
similarity = cosine_similarity(vectors)

In [48]:
def recommend(movies):
     movies_index =new[new['title'] == movies].index[0]
     distances = similarity[movies_index]
     movies_list = sorted(list(enumerate(distances)),reverse=True,key =lambda x:x[1])[1:6]
     for i in movies_list:
          print(new.iloc[i[0]].title)

In [49]:
recommend('Batman Begins')

The Dark Knight
The Dark Knight Rises
Amidst the Devil's Wings
Mi America
Ajami


In [50]:
import pickle

In [51]:
pickle.dump(new,open('movies5.pkl','wb'))

In [52]:
new.head()

,id,title,tags
0,19995,Avatar,"I n t h e 2 2 n d c e n t u r y , a p a r a p ..."
1,285,Pirates of the Caribbean: At World's End,"C a p t a i n B a r b o s s a , l o n g b e l ..."
2,206647,Spectre,A c r y p t i c m e s s a g e f r o m B o n d ...
3,49026,The Dark Knight Rises,F o l l o w i n g t h e d e a t h o f D i s t ...
4,49529,John Carter,"J o h n C a r t e r i s a w a r - w e a r y , ..."
